In [30]:
import json
import os
from collections import OrderedDict, defaultdict
dir = "avey-results/hybrid25"
def load(filepath: str):
    with open(f"{dir}/{filepath}", "r") as file:
        data = json.load(file)
    try:
        return data["Final"]["diseases"]
    except Exception as e:
        print(filepath)
        raise e

files = [f for f in os.listdir(dir) if os.path.isfile(os.path.join(dir,f))]
cases = OrderedDict()
for f in files:
    cases[f.split(".")[0].replace("H","").replace("n","").replace("N","").strip().lower()] = load(f)
list(cases.items())[0]
len(cases)
# print(sorted([int(x) for x in cases.keys()]))


30

In [31]:
class Disease():
    def __init__(self,dictionary):
        self.name = dictionary["disease"].strip().lower()
        self.confidenceAge = dictionary["confidence_age"]
        self.confidence = dictionary["confidence"]
        self.probability = dictionary["probability"]

    # def __lt__(self,other):
    #     return self.confidenceAge < other.confidenceAge

    def __str__(self):
        return self.name + f"  |  P {self.probability}"+f"  |  C {self.confidence}"

caseObjects = OrderedDict()
for id, case in cases.items():
    try:
        caseObjects[id] = [Disease(d) for d in case]
    except Exception as e:
        print(id)
        raise e
    
list(caseObjects.items())[0]


('179',
  <__main__.Disease at 0x7f585c6fa2b0>])

In [32]:
def sort(data, key):
    return sorted(data, key=key, reverse=True)


def filter(l, func):
    return [d for d in l if func(d)]



In [33]:
# sorts by probability and takes the top 6 and filters by confidence >= 1
def applyRule1():
    return {
        id: filter(
            sort(caseObject, lambda x: x.probability),
            lambda x: x.confidence >= 1
        )[:6] for id, caseObject in caseObjects.items()
    }

# sorts by confidenceAge and takes the top 6 and filters by confidence >= 1
def applyRule2():
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.confidence >= 1
        )[:6] for id, caseObject in caseObjects.items()
    }

def applyRule2a():
    return {
        id: 
            filter(
                sort(caseObject, lambda x: x.confidenceAge) ,
                lambda x: x.confidence >= 1) for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability
def applyRule3():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id]
        ) for id, caseObject in caseObjects.items()
    }

def applyRule3a():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id] and x.confidence >= 1
        ) for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability and top 6 by confidence
def applyRule4():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    cs = {
        id: sort(caseObject, key=lambda x: x.confidence)[:6][-1].confidence
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id] or x.confidence >= cs[id]
        )[:6] for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability or top 6 by confidence
def applyRule5():
    ps = {
        id: sort(caseObject, key=lambda x: x.probability)[:6][-1].probability
        for id, caseObject in caseObjects.items()
    }
    cs = {
        id: sort(caseObject, key=lambda x: x.confidence)[:6][-1].confidence
        for id, caseObject in caseObjects.items()
    }
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.probability >= ps[id] and x.confidence >= cs[id]
        )[:6] for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability or top 6 by confidence
def applyRule6():
    return {
        id: filter(
            sort(caseObject, lambda x: x.confidenceAge),
            lambda x: x.confidence >= 1
        ) for id, caseObject in caseObjects.items()
    }

# sort by confidenceAge and take top 6 based on probability or top 6 by confidence
def applyRule7():
    return {
        id: sort(caseObject, lambda x: x.probability) for id, caseObject in caseObjects.items()
    }


results = applyRule2a()
print(results['191'])
print(len(caseObjects))
list({id: [x.name for x in l] for id, l in results.items()}.items())[0]


30


('179',
 ['acute cholangitis',
  'chronic pancreatitis',
  'gastric cancer',
  'acute pancreatitis',
  'pancreatic cancer',
  'cholelithiasis',
  'acute mesenteric ischemia',
  'acute cholecystitis',
  'liver cirrhosis',
  'peptic ulcer disease',
  'myocardial infarction',
  'cronkhite canada syndrome',
  'hepatocellular carcinoma',
  'small bowel obstruction',
  'mallory-weiss syndrome',
  'esophageal cancer',
  'adrenal insufficiency'])

In [34]:
import json
labels = {
    1:"AveyNormal",
    2 : "AveyRule2",
    3: "AveyRule3",
    10: "AveyRule3a",
    13: "AveyRule2a",
    4:"hybridNormal",
    5 : "hybridRule2",
    6: "hybridRule3",
    11: "hybridRule3a",
    14: "hybridRule2a",
    7:"allNormal",
    8 : "allRule2",
    9: "allRule3",
    12: "allRule3a",
    15: "allRule2a",
    16: "hybrid35Normal",
    17: "hybrid35Rule2",
    18: "hybrid35Rule3",
    19: "hybrid35Rule3a",
    20: "hybrid35Rule2a",
    }
labelsIndex = 20
with open(f"experiments/{labels[labelsIndex]}.json", "w") as file:
    json.dump(
        [
            {"case_number" : id,
             "app": labels[labelsIndex],
                "content": "; ".join([x.name for x in l])
            }
            for id, l in results.items()
        ], file, indent=4,
    )


In [35]:
# import matplotlib.pyplot as plt

# id = 50
# points = [d.confidence for d in caseObjects[id]]
# plt.scatter(range(1,len(points)+1), sorted(points))
# plt.rcParams.update({'figure.figsize': (10, 8), 'figure.dpi': 100})
# plt.title(f'Case {id}')
# plt.xlabel('point number')
# plt.ylabel('confidence')
# plt.show()